<a href="https://colab.research.google.com/github/saagi/Helloworld/blob/master/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq >> /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
!ls /usr/lib/jvm

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64


sample_data


In [ ]:
!pip install -U pyarrow

In [3]:
import os
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"]="/content/spark-2.4.5-bin-hadoop2.7"

In [4]:
import findspark
findspark.init()

In [5]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.executor.memory","8g")
spark.conf.set("spark.driver.memory","2g")
spark.conf.set("spark.memory.fraction","0.9")

In [6]:
!gdown --id 1v0tvhXh61I0I48IdLE_gG175pdgRfUhV

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1v0tvhXh61I0I48IdLE_gG175pdgRfUhV
To: /content/Technical_Round.zip
100% 18.8M/18.8M [00:00<00:00, 36.1MB/s]


In [7]:
!unzip '/content/Technical_Round.zip'

Archive:  /content/Technical_Round.zip
  inflating: Technical_Round_v2/material_list.txt  
  inflating: Technical_Round_v2/Data.csv  


In [8]:
df = spark.read.options(header='true', inferSchema='true').csv("Technical_Round_v2/Data.csv")
df = df.limit(20000)

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv -P sample_data
# pyspark practise





In [28]:
df=spark.read.options(header=True,delimeter=';').csv("sample_data/winequality-red.csv")

In [ ]:
df.printSchema()
df.show(5,False)

In [10]:
from pyspark.sql.functions import col,lit,when

In [11]:
#df.where(col("cancel_status")!=1).show(5,False)
df.select(col("name_1").alias("Name"),col("uom"),col("postal_code")).show(5,False)
#df.selectExpr("name_1 as Name","uom","postal_code as PIN_code").show(5,False)
#df.select(df.columns).show(4,False)
#len(df.columns)

+----------------------+---+-----------+
|Name                  |uom|postal_code|
+----------------------+---+-----------+
|o vidhya sagar        |KG |501401     |
|Anushka fresh world   |KG |560067     |
|Naveen provision store|KG |560035     |
|Greens vegetables     |KG |562125     |
|Anushka fresh world   |KG |560067     |
+----------------------+---+-----------+
only showing top 5 rows



In [61]:
df2=df.select("name_1").distinct()
df3=df.join(df2,df.name_1==df.name_1,"inner")
#df3.count()
#df.select("name_1").where("name_1 is null").count()
#df.dropDuplicates(["name_1","cancel_status"]).count()


In [ ]:
#df.where("delivery_date is not null").orderBy(col("delivery_date").desc()).show(4,False)
#df.where("delivery_date is not null").orderBy(col("delivery_date").desc()).show(4,False)
df.orderBy(col("delivery_date").desc_nulls_first()).show(4,False)

In [ ]:
df.createOrReplaceTempView("temp")
spark.sql("""
select * from temp where delivery_date is not null and cancel_status=1 order by delivery_date desc
""").show(5,False)

In [ ]:
from pyspark.sql.functions import min,max,sum,count
#df.groupBy("cancel_status").count().show(5,False) -> will give overall count
#df.groupBy("cancel_status").max().show(3,False)  -> will give max of each column
df.groupBy("cancel_status").agg(sum("order_quantity").alias("sum_quantity"),count("order_quantity").alias("count_orders")).show(10,False)



In [13]:
#Joins
df1=df.select("name_1","cancel_status").distinct()
df2=df.select("name_1").distinct()
df3=df1.join(df2,df1.name_1==df2.name_1,"outer")

#df3.show(5,False)
df1.join(df2,df1.name_1==df2.name_1,"leftsemi").show(5,False)

+----------------------+-------------+
|name_1                |cancel_status|
+----------------------+-------------+
|o vidhya sagar        |1            |
|Anushka fresh world   |1            |
|Naveen provision store|1            |
|Greens vegetables     |1            |
|r s provision store   |1            |
+----------------------+-------------+
only showing top 5 rows



In [24]:
## PartitionBy and repartitions..
# partitionBy("<Column>") -> creates each sub directory wrt each partition <Column>
# repartition(k) -> in each sub directory there are k files each representing each partition

df.rdd.getNumPartitions()
df=df.repartition(10)
df.rdd.getNumPartitions()
df.write.partitionBy("cancel_status").options(header=True).mode("overwrite").csv("/tmp/dfpart")

In [26]:
!ls /tmp/dfpart/'cancel_status=1'

part-00000-9fb8e61a-df2c-4bd5-ab63-622193a5dbb1.c000.csv
part-00001-9fb8e61a-df2c-4bd5-ab63-622193a5dbb1.c000.csv
part-00002-9fb8e61a-df2c-4bd5-ab63-622193a5dbb1.c000.csv
part-00003-9fb8e61a-df2c-4bd5-ab63-622193a5dbb1.c000.csv
part-00004-9fb8e61a-df2c-4bd5-ab63-622193a5dbb1.c000.csv
part-00005-9fb8e61a-df2c-4bd5-ab63-622193a5dbb1.c000.csv
part-00006-9fb8e61a-df2c-4bd5-ab63-622193a5dbb1.c000.csv
part-00007-9fb8e61a-df2c-4bd5-ab63-622193a5dbb1.c000.csv
part-00008-9fb8e61a-df2c-4bd5-ab63-622193a5dbb1.c000.csv
part-00009-9fb8e61a-df2c-4bd5-ab63-622193a5dbb1.c000.csv


In [ ]:
df=df.select('delivery_date', 'cancel_status', 'material_description', 'material_no', 'plant', 'distribution_channel', 'uom','order_quantity')

In [ ]:
train_df,test_df=df.randomSplit([0.8,0.2],seed=12)
train_df.count(),test_df.count()

(16045, 3955)

In [ ]:
train_df.where("delivery_date like '2021-%' and uom is not null").show(5,False)

+-------------+-------------+---------------------+------------------+-----+--------------------+---+--------------+
|delivery_date|cancel_status|material_description |material_no       |plant|distribution_channel|uom|order_quantity|
+-------------+-------------+---------------------+------------------+-----+--------------------+---+--------------+
|2021-11-19   |1            |APPLE GALA           |WC0000000000000161|1009 |52                  |KG |2.0           |
|2021-11-19   |1            |APPLE GALA 4PC - PACK|WC0000000000000013|1009 |52                  |PAC|2.0           |
|2021-11-19   |1            |APPLE GALA 4PC - PACK|WC0000000000000013|1009 |52                  |PAC|4.0           |
|2021-11-19   |1            |APPLE GALA 4PC - PACK|WC0000000000000013|1009 |52                  |PAC|4.0           |
|2021-11-19   |1            |APPLE GALA 4PC - PACK|WC0000000000000013|1009 |52                  |PAC|5.0           |
+-------------+-------------+---------------------+-------------

In [ ]:
df.count()

709334

In [ ]:
df=df.select('delivery_date', 'cancel_status', 'material_description', 'material_no', 'plant', 'distribution_channel', 'uom','order_quantity')

In [ ]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[delivery_date: string, cancel_status: string, material_description: string, material_no: string, plant: int, distribution_channel: int, uom: string, order_quantity: double]>

In [ ]:
df.where("material_description like '%BHENDI%' and plant='1000' and delivery_Date>='2021-08-16 and delivery_Date<=2021-11-25'").orderBy("delivery_date").show(5,False)

+-------------+-------------+----------------------+------------------+-----+--------------------+---+--------------+
|delivery_date|cancel_status|material_description  |material_no       |plant|distribution_channel|uom|order_quantity|
+-------------+-------------+----------------------+------------------+-----+--------------------+---+--------------+
|2021-08-17   |1            |BHENDI (LADIES FINGER)|WC0000000000230022|1000 |50                  |KG |3.0           |
|2021-08-17   |1            |BHENDI (LADIES FINGER)|WC0000000000230022|1000 |50                  |KG |6.0           |
|2021-08-17   |1            |BHENDI (LADIES FINGER)|WC0000000000230022|1000 |51                  |KG |5.0           |
|2021-08-17   |1            |BHENDI (LADIES FINGER)|WC0000000000230022|1000 |50                  |KG |3.0           |
|2021-08-17   |1            |BHENDI (LADIES FINGER)|WC0000000000230022|1000 |50                  |KG |3.0           |
+-------------+-------------+----------------------+----

In [ ]:
df.where("material_description like '%BHENDI%' and plant='1000' and delivery_Date>='2021-08-16 and delivery_Date<=2021-11-25'").groupBy("delivery_Date").count().orderBy("delivery_Date").show(15,False)

+-------------+-----+
|delivery_Date|count|
+-------------+-----+
|2021-08-17   |28   |
|2021-08-18   |23   |
|2021-08-19   |18   |
|2021-08-20   |26   |
|2021-08-21   |21   |
|2021-08-22   |14   |
|2021-08-23   |7    |
|2021-08-24   |27   |
|2021-08-25   |10   |
|2021-08-26   |21   |
|2021-08-27   |21   |
|2021-08-28   |15   |
|2021-08-29   |30   |
|2021-08-30   |11   |
|2021-08-31   |34   |
+-------------+-----+
only showing top 15 rows



In [ ]:
df.where("delivery_Date like '2021%'").selectExpr("min(delivery_Date) as min_date").show(2,False)

+----------+
|min_date  |
+----------+
|2021-07-01|
+----------+



In [ ]:
df.selectExpr("max(delivery_Date) as max_date").show(2,False)

+----------+
|max_date  |
+----------+
|2021-11-25|
+----------+



In [ ]:
df.rdd.getNumPartitions()

2

In [ ]:
df=df.repartition(10)
df.rdd.getNumPartitions()

10

In [ ]:
df=df.coalesce(5)
df.rdd.getNumPartitions()

5

In [ ]:
from pyspark.sql.functions import desc,sum,col
df2=df.groupBy("material_no").count().orderBy(desc("count"))

In [ ]:
df3=df.select("material_description","material_no").distinct()

AnalysisException: ignored